In [1]:
drive_path = '/content/drive'
from google.colab import drive
drive.mount(drive_path)


Mounted at /content/drive


In [2]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm

In [19]:
demographic = pd.read_csv("/content/drive/MyDrive/Data/demographic.csv")
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
column_list = pd.read_csv("/content/drive/MyDrive/Data/columnLabels.csv").columns
electrodes= list(pd.read_csv('/content/drive/MyDrive/Data/columnLabels.csv').columns[4:])

N_PERSON = 81
N_AVERAGED = 16
x_count = 0
num_electrodes = len(electrodes)
num_rows = 81 * 100
num_cols = 9216 * (num_electrodes // N_AVERAGED)
X = np.zeros((num_rows, num_cols), dtype=np.float32)
Y = np.zeros(len(X))

In [17]:
def averaged_by_N_rows(a, n):
    assert a.ndim == 2, "Input matrix must be 2-dimensional"
    assert a.shape[0] % n == 0, "Number of rows in the input matrix must be divisible by n"
    return np.mean(a.reshape(-1, n, a.shape[1]), axis=1)

In [20]:

for i in tqdm(range(N_PERSON)):

    person = str(i+1)
    csv_path = f"{'/content/drive/MyDrive/Data'}/{person}.csv/{person}.csv"
    df = pd.read_csv(csv_path, header=None, names=column_list)
    grouped_trials = df.groupby("trial")
    
    for trial_number, trial_data in grouped_trials:
        number_of_trials = len(trial_data)
        if number_of_trials == 9216:
            current_sample_matrix = trial_data[electrodes].values
            averaged_by_N = averaged_by_N_rows(current_sample_matrix, n=N_AVERAGED)
            averaged_by_N_big_vec = averaged_by_N.reshape(-1)
            np.append(X,averaged_by_N_big_vec.astype(np.float32))
            np.append(Y,diagnosis_dict[int(person)])

  0%|          | 0/81 [00:00<?, ?it/s]

In [23]:
np.save('/content/drive/MyDrive/Data/X.npy', X);
np.save('/content/drive/MyDrive/Data/Y.npy', Y);

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)
X_norm = (normalize(X.reshape(-1, 70), axis=0, norm='max')).reshape(X.shape)
X_train_norm, X_test_norm, Y_train_norm, Y_test_norm = train_test_split(X_norm, Y, test_size=0.2, shuffle=True, random_state=42)
_norm = X
X_train_2d = X_train_norm.reshape(X_train_norm.shape[0], len(electrodes), X_train_norm.shape[1] // len(electrodes), 1)
X_test_2d = X_test_norm.reshape(X_test_norm.shape[0], len(electrodes), X_test_norm.shape[1] // len(electrodes), 1)